In [1]:
from pathlib import Path
import xarray as xr

In [ ]:
# traverse path and process files

input_dir = Path('/home/ubuntu/tide_models/fes2022b')
output_dir = Path('/home/ubuntu/tide_models_clipped/fes2022b')

def _fix_encoding(encoding: dict) -> dict:
    good_keys = ["shuffle", "zlib", "complevel", "dtype", "_FillValue"]
    return {k: v for k, v in encoding.items() if k in good_keys}

for path in input_dir.glob("*/*.nc"):
    with open(path, "rb") as nc:
        print(f"Clipping {path}")
        src = xr.open_dataset(nc, engine="h5netcdf")
        output = src.where(
             (src.lat > -11) & (src.lat < 6) & (src.lon > 94) & (src.lon < 142),
             drop=True,
         )

        subdir = path.parent.relative_to(input_dir)
        output_filename = f"{path.with_suffix('').stem}.nc"

        full_output_dir = output_dir / subdir
        full_output_dir.mkdir(parents=True, exist_ok=True)

        output_path = f"{full_output_dir}/{output_filename}"


        encoding = {var: _fix_encoding(src[var].encoding) for var in src}
        output.to_netcdf(
            output_path,
            encoding=encoding,
            engine="h5netcdf",
        )

